# 第31章 类的高级主题

## 扩展内置类型

- 除了实现新的种类的对象以外，类偶尔也用于扩展Python的内置类型的功能，从而支持更另类的数据结构

### 通过嵌入扩展类型

- 写一个类，包装内置类型对象，并提供一些扩展方法以供调用

### 通过子类扩展类型

- 从Python2.2起，所有内置类型都能直接创建子类，像list、str、dict以及tuple这些类型转换函数都变成内置类型的名称：虽然脚本看不见，但类型转换调用，例如，list('spam')其实是启用了类型的对象构造函数
- 这样的改变让你可以通过用户定义的class语句，定制或扩展内置类型的行为：建立类型名称的子类并对其进行定制。类型的子类实例，可用在原始的内置类型能够出现的任何地方
- 一般来说，用这种方式定制内置类型，是很强大的

In [1]:
class MyList(list):
    def __getitem__(self, offset):
        print "indexing %s at %s" % (self, offset)
        return list.__getitem__(self, offset - 1)
print list('abc')
x = MyList('abc')
print x[1]
print x[3]
x.append('spam')
print x
x.reverse()
print x

['a', 'b', 'c']
indexing ['a', 'b', 'c'] at 1
a
indexing ['a', 'b', 'c'] at 3
c
['a', 'b', 'c', 'spam']
['spam', 'c', 'b', 'a']


## 新式类

- 对于Python3.0来说，所有的类都是所谓的“新式类”，它们都继承自object，无论是显式还是隐式，并且，所有的对象都是object的实例
- 在Python2.6及其以前的版本中，类必须显式继承自object，才能看做新式类，并获得新式类的特性
- 通常情况下，任何从object或其它内置类型派生的类，都会自动视为新式类，只要一个内置类型位于超类树中的某个位置，新类也当做一个新式类；不是从内置类型派生出来的类，就会当做经典类来对待

### 新式类变化

- 这里我们只介绍一点，就是在新式类中，类和类型合并了：type(I)返回一个实例所创建自的类，而不是一个通用的instance，并且，通常和`I.__class__`是一致的

In [2]:
type(x)

__main__.MyList

In [3]:
x.__class__

__main__.MyList

In [4]:
type(MyList)

type

In [5]:
MyList.__class__

type

In [6]:
class Test:
    pass
y = Test()
type(y)

instance

In [7]:
type(Test)

classobj

In [8]:
# 以下语句报错
#Test.__class__

- 在Python3.0中，类型自身派生自object，并且object派生自type

### 钻石继承变动

- 对经典类而言，继承搜索顺序是绝对深度优先，然后才是由左至右；在新式类中，搜索相对来说是宽度优先的

In [9]:
# 搜索顺序：DBAC
class A:
    attr = 1
class B(A):
    pass
class C(A):
    attr = 2
class D(B, C):
    pass
x = D()
x.attr

1

In [10]:
# 搜索顺序：DBCA
class A(object):
    attr = 1
class B(A):
    pass
class C(A):
    attr = 2
class D(B, C):
    pass
x = D()
x.attr

2

## 新式类的扩展

### slots实例

- 将字符串属性名称顺序赋值给特殊的`__slots__`类属性，新式类就有可能既限制类的实例将有的合法属性集，又能够优化内存和速度性能
- 这个特殊属性一般是在class语句顶层内将字符串名称顺序赋值给变量`__slots__`而设置：只有`__slots__`列表内的这些变量名可赋值为实例属性，不过，实例属性名必须在引用前赋值

In [11]:
class limiter(object):
    __slots__ = ['age', 'name', 'job']
x = limiter()
x.age # 必须先赋值

AttributeError: age

In [12]:
x.age = 40
x.age

40

In [13]:
x.ape # 该属性不存在于__slots__中

AttributeError: 'limiter' object has no attribute 'ape'

In [14]:
x.__dict__

AttributeError: 'limiter' object has no attribute '__dict__'

## 静态方法和类方法

- 在Python2.X中，有可能在类中定义两种方法，它们不用一个实例就可以调用：静态方法和类方法，前者大致与一个类中的简单的无实例函数类似地工作，后者传递一个类而不是一个实例

### 为什么使用特殊方法

- 有时候程序需要处理与类而不是与实例相关的数据，为此，我们需要一个类中的方法不仅不传递而且也不期待一个self实例参数，Python通过静态方法的概念来支持这样的目标——嵌套在一个类中的没有self参数的简单函数，并且旨在操作类属性而不是实例属性

### 使用静态方法和类方法

In [15]:
class Methods:
    # 实例方法
    def imeth(self, x):
        print self, x
    # 静态方法
    def smeth(x):
        print x
    # 类方法
    def cmeth(cls, x):
        print cls, x
    smeth = staticmethod(smeth)
    cmeth = classmethod(cmeth)

In [16]:
obj = Methods()
obj.imeth(1)
Methods.imeth(obj, 2)

<__main__.Methods instance at 0x10e9b8830> 1
<__main__.Methods instance at 0x10e9b8830> 2


In [17]:
Methods.smeth(3)
obj.smeth(3)

3
3


In [18]:
# 把类（而不是实例）传入类方法第一个参数中，无论通过类或者实例去调用
Methods.cmeth(4)
obj.cmeth(4)

__main__.Methods 4
__main__.Methods 4


### 使用静态方法统计实例

In [19]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances += 1
    def printNumInstances():
        print "Number of instances: ", Spam.numInstances
    printNumInstances = staticmethod(printNumInstances)
a = Spam()
b = Spam()
c = Spam()
Spam.printNumInstances()
a.printNumInstances()

Number of instances:  3
Number of instances:  3


- 类可以继承静态方法而不用重新定义它，它可以没有一个实例而运行，不管定义于类树的何处

In [20]:
class Other(Spam):
    pass
c = Other()
c.printNumInstances()

Number of instances:  4


### 用类方法统计实例

In [21]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances += 1
    def printNumInstances(cls):
        print "Number of instances: ", cls.numInstances, cls
    printNumInstances = classmethod(printNumInstances)
a = Spam()
b = Spam()
c = Spam()
Spam.printNumInstances()
a.printNumInstances()

Number of instances:  3 __main__.Spam
Number of instances:  3 __main__.Spam


- 类方法接收调用的主体是最底层的类

In [22]:
class Sub(Spam):
    def printNumInstances(cls):
        print "Extra stuff...", cls
        Spam.printNumInstances()
    printNumInstances = classmethod(printNumInstances)
class Other(Spam):
    pass
x = Sub()
y = Spam()
x.printNumInstances()
Sub.printNumInstances()
y.printNumInstances()
z = Other()
z.printNumInstances()

Extra stuff... __main__.Sub
Number of instances:  5 __main__.Spam
Extra stuff... __main__.Sub
Number of instances:  5 __main__.Spam
Number of instances:  5 __main__.Spam
Number of instances:  6 __main__.Other


- 静态方法和显式类名称可能对于处理一个类本地的数据来说是更好的解决方案
- 类方法可能更适合处理对层级中的每个类不同的数据

In [23]:
class Spam:
    numInstances = 0
    def count(cls):
        cls.numInstances += 1
    def __init__(self):
        self.count()
    count = classmethod(count)

class Sub(Spam):
    numInstances = 0
    def __init__(self):
        Spam.__init__(self)
        
class Other(Spam):
    numInstances = 0
    
x = Spam()
y1, y2 = Sub(), Sub()
z1, z2, z3 = Other(), Other(), Other()
x.numInstances, y1.numInstances, z1.numInstances

(1, 2, 3)

In [24]:
Spam.numInstances, Sub.numInstances, Other.numInstances

(1, 2, 3)

## 装饰器和元类：第一部分

- 函数装饰器（function decorator）提供了一种方式，替函数明确了特定的运算模式，也就是将函数包裹了另一层，在另一函数的逻辑内实现

### 函数装饰器基础

- 从语法上来讲，函数装饰器是它后边的函数的运行时的声明，函数装饰器是写成一行，就在定义函数或方法的def语句之前，而且由@符号、后面跟着所谓的元函数（metafunction）组成：也就是管理另一函数（或其它可调用对象）的函数
- 装饰器函数可返回原始函数，或者新对象（保存传给装饰器的原始函数，这个函数将会在额外逻辑层执行后间接地运行）

In [25]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    @staticmethod
    def printNumInstances():
        print "Number of instances created: ", Spam.numInstances
a = Spam()
b = Spam()
c = Spam()
Spam.printNumInstances()
a.printNumInstances()

Number of instances created:  3
Number of instances created:  3


- Python提供一些内置函数装饰器，来做一些运算，例如，标识静态方法，但是我们也可以编写自己的任意装饰器，虽然不限于使用类，但用户定义的函数装饰器通常也写成类，把原始函数和其它数据当成状态信息
- staticmethod是一个内置函数，它可用于装饰语法中，只是因为它把一个函数当做参数并且返回一个可调用对象，实际上任何这样的函数都可以以这种方式使用，包括用户自定义函数

### 装饰器例子

In [26]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args):
        self.calls += 1
        print "call %s to %s" % (self.calls, self.func.__name__)
        self.func(*args)

@tracer # 等同于 spam = tracer(spam)
def spam(a, b, c):
    print a, b, c
    
spam(1, 2, 3)
spam('a', 'b', 'c')
spam(4, 5, 6)

call 1 to spam
1 2 3
call 2 to spam
a b c
call 3 to spam
4 5 6


### 类装饰器和元类

- 类装饰器类似于函数装饰器